In [2]:
import pandas as pd
import altair as alt
import duckdb

from altair import datum

In [3]:
df = pd.read_csv('lobbywatch_medienkonferenz.csv')
df_bekannt = duckdb.query('select * from df where bezahlt is not null').to_df()
df_bezahlt = duckdb.query('select * from df where bezahlt = \'ja\'').to_df()
df_fraktionen = duckdb.query('''
select 'G' fraktion, 'Grüne' fraktion_lang, '#85B50C' farbe, 1 sort
union
select 'S' fraktion, 'SP' fraktion_lang, '#E4022D' farbe, 2 sort
union
select 'GL' fraktion, 'GLP' fraktion_lang, '#A0A000' farbe, 3 sort
union
select 'M-E' fraktion, 'Mitte' fraktion_lang, '#FF9100' farbe, 4 sort
union
select 'RL' fraktion, 'FDP' fraktion_lang, '#3A8BC1' farbe, 5 sort
union
select 'V' fraktion, 'SVP' fraktion_lang, '#0A7228' farbe, 6 sort
''').to_df()

fraktion_dict = {
    'Grüne': '#85B50C',
    'SP': '#E4022D',
    'Mitte': '#FF9100',
    'GLP': '#A0A000',
    'FDP': '#3A8BC1',
    'SVP': '#0A7228',
}
# for p in dict(fraktion_dict).keys():
#     if p not in source['fraktion'].values:
#         del fraktion_dict[p]

fraktion_namen, fraktion_farben = list(fraktion_dict.keys()), list(fraktion_dict.values())

# Untersuchte Interessenbindungen
Untersucht wurden alle Interessnbindungen, welche folgenden Kriterien entsprechen:
- Sind gemäss Parlamentariergesetz (ParlG) Art. 11 deklarationspflichtig
- Werden von einem gegenwärtig amtierenden Parlamentarier ausgeübt
- Es handelt sich hierbei nicht um eine hauptberufliche Tätigkeit
- Die Interessenbindung wurde gemäss dem Prozess von Lobbywatch recherchiert, kontrolliert und freigegeben
- Die Organisation ist keine Partei, parlamentarische Gruppe, parlamentarische Freundschaftsgruppe oder eine ausserparlamentarische Kommission

# Kennzahlen
Schauen wir uns zuerst ein paar Kennzahlen an

In [291]:
duckdb.query('''
select 'total' kennzahl, (select count(1) from df_bekannt) wert, 100.0*(select count(1) from df_bekannt)/(select count(1) from df_bekannt) anteil
union
select 'deklariert', (select count(1) from df_bekannt where deklariert = 'ja'), 100.0*(select count(1) from df_bekannt where deklariert = 'ja')/(select count(1) from df_bekannt)
union
select 'bezahlt', (select count(1) from df_bekannt where bezahlt = 'ja'), 100.0*(select count(1) from df_bekannt where bezahlt = 'ja')/(select count(1) from df_bekannt where bezahlt is not null)
union
select 'mit angabe zur vergütung', (select count(1) from df_bekannt where verguetung is not null), 100.0*(select count(1) from df_bekannt where verguetung is not null)/(select count(1) from df_bekannt)
union
select 'bezahlt aber undeklariert', (select count(1) from df_bekannt where bezahlt = 'ja' and deklariert = 'nein'), 100.0*(select count(1) from df_bekannt where bezahlt = 'ja' and deklariert = 'nein')/(select count(1) from df_bekannt)
''').to_df()

,kennzahl,wert,anteil
0,total,2070,100.000000
1,deklariert,1749,84.492754
2,bezahlt,838,40.483092
3,mit angabe zur vergütung,253,12.222222
4,bezahlt aber undeklariert,45,2.173913


Bezahlte Interessenbindungen aufgeschlüsselt nach Branche.

In [292]:
source = duckdb.query('''
    select branche, count(1) count
    from df_bezahlt
    group by 1
''').to_df()

chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anzahl'),
    y=alt.Y('branche', title='Branche'),
    tooltip='count'
)
chart.save('output/bars-nach-branche-absolut.pdf')
chart

alt.Chart(...)

In [293]:
source = duckdb.query('''
    select fraktion_lang fraktion, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    group by 1
''').to_df()

bars = alt.Chart(source).mark_bar(opacity=1.0).encode(
    x=alt.X('count', title='Anzahl', stack=None),
    y=alt.Y('fraktion', sort='-x', title='Fraktion'),
    color=alt.Color('fraktion', scale=alt.Scale(domain=fraktion_namen, range=fraktion_farben), legend=None),
    tooltip='count'
)

text = alt.Chart(source).mark_text(dx=-15, color='white').encode(
    x=alt.X('count', title='Anzahl', stack=None),
    y=alt.Y('fraktion', sort='-x', title='Fraktion'),
    detail='fraktion',
    text='count',
)

chart = bars + text
chart.save('output/bars-nach-fraktion-absolut.pdf')
chart

alt.LayerChart(...)

In [294]:
source = duckdb.query('''
    select f.fraktion_lang fraktion, 'bezahlt' bezahlt, 1.0 * count(case bezahlt when 'ja' then 1 end) / count(1) count, 1.0 * count(case bezahlt when 'ja' then 1 end) / count(1) sort
    from df_bekannt b
    join df_fraktionen f on f.fraktion = b.fraktion
    group by 1, 2
    union all
    select f.fraktion_lang fraktion, 'unbezahlt' bezahlt, 1.0 * count(case bezahlt when 'nein' then 1 end) / count(1) count, 1.0 * count(case bezahlt when 'ja' then 1 end) / count(1) sort
    from df_bekannt b
    join df_fraktionen f on f.fraktion = b.fraktion
    group by 1, 2
''').to_df()

bars = alt.Chart(source).mark_bar(opacity=1.0).encode(
    x=alt.X('count', title='Anteil', sort='-x', stack='zero', axis=alt.Axis(format='%')),
    y=alt.Y('fraktion', sort=alt.EncodingSortField('sort', order='descending'), title='Fraktion'),
    color='bezahlt',
    tooltip='count',
)

text = alt.Chart(source).mark_text(dx=-20, color='white').encode(
    x=alt.X('count', title='Anteil', stack='zero'),
    y=alt.Y('fraktion', sort=alt.EncodingSortField('sort', order='descending'), title='Fraktion'),
    detail='bezahlt',
    text=alt.Text('count', format='.1%'),
)

chart = bars + text
chart.save('output/bars-nach-fraktion-und-bezahlt-relativ.pdf')
chart

alt.LayerChart(...)

In [295]:
source = duckdb.query('''
    select fraktion_lang fraktion, 'bezahlt' bezahlt, count(case bezahlt when 'ja' then 1 end) count, count(case bezahlt when 'ja' then 1 end) sort
    from df_bekannt b
    join df_fraktionen f on f.fraktion = b.fraktion
    group by 1, 2
    union all
    select fraktion_lang fraktion, 'unbezahlt' bezahlt, count(case bezahlt when 'nein' then 1 end), count(case bezahlt when 'ja' then 1 end)
    from df_bekannt b
    join df_fraktionen f on f.fraktion = b.fraktion
    group by 1, 2
''').to_df()

bars = alt.Chart(source).mark_bar(opacity=1.0).encode(
    x=alt.X('count', title='Anzahl', sort='-x', stack='zero'),
    y=alt.Y('fraktion', sort=alt.EncodingSortField('sort', order='descending'), title='Fraktion'),
    color='bezahlt',
    tooltip='count',
)

text = alt.Chart(source).mark_text(dx=-20, color='white').encode(
    x=alt.X('count', title='Anzahl', stack='zero'),
    y=alt.Y('fraktion', sort=alt.EncodingSortField('sort', order='descending'), title='Fraktion'),
    detail='bezahlt',
    text=alt.Text('count'),
)

chart = bars + text
chart.save('output/bars-nach-fraktion-und-bezahlt-absolut.pdf')
chart

alt.LayerChart(...)

Hier sieht man wie sich die Fraktionen die bezahlten Mandate aufteilen innerhalb der Branchen.
Auffällig ist, dass die linken Parteien nur in der Umweltbranche eine Mehrheit der Lobbymandate haben (falls man die GLP zu den linken zählt). Hingegen sind sie in den Branchen Landwirtschaft, Verkehr und Wirtschaft sehr schwach vertreten.

In [296]:
source = duckdb.query('''
    select branche, fraktion_lang fraktion, f.sort, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    where 1=1
    and branche in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2, 3
    union
    select 'Übrige' branche, fraktion_lang fraktion, sort, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    where branche not in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2, 3
    order by branche, f.sort
''').to_df()

chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anteil', stack='normalize', axis=alt.Axis(format='%')),
    y=alt.Y('branche', title='Branche'),
    color=alt.Color('fraktion', scale=alt.Scale(domain=fraktion_namen, range=fraktion_farben)),
    order=alt.Order('color_fraktion_sort_index:Q'),
)
chart.save('output/bars-nach-branche-und-fraktion-relativ-min20.pdf')
chart

alt.Chart(...)

Umgekehrt kann man auch anschauen wo die Schwerpunkte der einzelnen Fraktionen liegen.
_Berücksichtigt hier sind alle Branchen mit insgesamt mindestens 20 bezahlten Lobbymandaten._

In [297]:
source = duckdb.query('''
    select branche, fraktion_lang fraktion, sort, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    where branche in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2, 3
    union
    select 'Übrige' branche, fraktion_lang fraktion, sort, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    where branche not in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2, 3
    order by branche, f.sort
''').to_df()

chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anteil', axis=alt.Axis(format='%'), stack='normalize'),
    y=alt.Y('fraktion', title='Fraktion', sort=fraktion_namen),
    color='branche',
).properties(
    # height=400
)
chart.save('output/bars-nach-fraktion-und-branche-relativ-min20.pdf')
chart

alt.Chart(...)

In [298]:
source = duckdb.query('''
    select branche, fraktion_lang fraktion, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    where branche in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2
    union
    select 'Übrige' branche, fraktion_lang fraktion, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    where branche not in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2
''').to_df()

chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anzahl'),
    y=alt.Y('fraktion', sort=['G','S','GL','M-E','RL','V'], title='Partei'),
    color=alt.Color('fraktion', scale=alt.Scale(domain=fraktion_namen, range=fraktion_farben)),
).properties(
    width=250
).facet(
    facet='branche',
    columns=3
).resolve_scale(
    x='independent',
    # radius='shared',
)
chart.save('output/mehrere-bars-nach-branche-und-fraktion-absolut-min20.pdf')
chart

alt.FacetChart(...)

In [299]:
source = duckdb.query('''
    select branche, fraktion_lang fraktion, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    where 1=1
    and branche in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2
    union
    select 'Übrige' branche, fraktion_lang fraktion, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    where branche not in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2
''').to_df()

chart = alt.Chart(source).mark_arc(innerRadius=40, stroke='white', strokeWidth=1.5, cornerRadius=4).encode(
    theta=alt.Theta('count', stack='normalize'),
    color=alt.Color('branche'),
    tooltip='count',
).properties(
    width=150,
    height=150
).facet(
    facet='fraktion',
    columns=3
)
chart.save('output/mehrere-kreise-nach-fraktion-und-branche-relativ-min20.pdf')
chart

alt.FacetChart(...)

In [5]:
source = duckdb.query('''
    select distinct p.branche, fraktion_lang fraktion,
        count(1) over (partition by p.branche, p.fraktion) count,
        count(1) over (partition by p.branche) count_all
    from (
        select branche, fraktion
        from (select distinct fraktion from df_bezahlt) p
        cross join (select branche from df_bezahlt group by branche having count(1) >= 20 /*order by count(1) desc limit 9*/) b
    ) p left join df_bezahlt on (p.branche, p.fraktion) = (df_bezahlt.branche, df_bezahlt.fraktion)
    join df_fraktionen f on f.fraktion = p.fraktion
    where 1=1
    union all
    select distinct 'Übrige', fraktion_lang fraktion, count(1) over (partition by p.fraktion), count(1) over ()
    from df_bezahlt p
    join df_fraktionen f on f.fraktion = p.fraktion
    where branche not in (select branche from df_bezahlt group by branche having count(1) >= 20)
''').to_df()

base = alt.Chart().encode(
    theta=alt.Theta('fraktion', stack=True, scale=alt.Scale(domain=fraktion_namen, rangeMin=0)),
    radius=alt.Radius('count:Q', scale=alt.Scale(type='sqrt', rangeMax=75), stack=False),
    color=alt.Color('fraktion', scale=alt.Scale(domain=fraktion_namen, range=fraktion_farben)),
    tooltip=[alt.Tooltip('fraktion'), alt.Tooltip('count', title='anzahl'), alt.Tooltip('rel_count', type='quantitative', format='.1%', title='anteil')],
).properties(
    width=180,
    height=180
).transform_calculate(
    rel_count='datum.count / datum.count_all',
)

arcs = base.mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4,
    # tooltip=True
)

text = base.mark_text(radiusOffset=10).encode(
    text='label:N'
).transform_calculate(
    # label='datum.rel_count == 0 ? "" : format(datum.rel_count, ".1%")'
    label='datum.count == 0 ? "" : datum.count'
)

chart = alt.layer(arcs, text, data=source).facet(
    facet='branche',
    columns=4
).resolve_scale(
    radius='independent',
    # radius='shared',
)
chart.save('output/mehrere-radial-nach-branche-und-fraktion-min20.pdf')
chart

alt.FacetChart(...)

In [301]:
source = duckdb.query('''
    select distinct p.branche, fraktion_lang fraktion,
        count(b.interessenbindung_id) over (partition by p.branche, p.fraktion) count,
        count(b.interessenbindung_id) over (partition by p.branche) count_all
    from (
        select distinct branche, fraktion
        from (select fraktion from df_bezahlt) p
        cross join (select branche from df_bezahlt group by branche having count(1) >= 20 /*order by count(1) desc limit 9*/) b
    ) p left join df_bezahlt b on (p.branche, p.fraktion) = (b.branche, b.fraktion)
    join df_fraktionen f on f.fraktion = p.fraktion
    where 1=1
    union all
    select distinct 'Übrige', fraktion_lang fraktion, count(1) over (partition by p.fraktion), count(1) over ()
    from df_bezahlt p
    join df_fraktionen f on f.fraktion = p.fraktion
    where branche not in (select branche from df_bezahlt group by branche having count(1) >= 20)
''').to_df()

chart = alt.Chart(source).mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4
).encode(
    # theta=alt.Theta('branche', stack=True),
    # radius=alt.Radius('count:Q', scale=alt.Scale(type='sqrt', rangeMax=75), stack=False),
    theta=alt.Theta('branche', stack='center'),
    radius=alt.Radius('count', type='quantitative', scale=alt.Scale(type='sqrt', rangeMax=70), stack=False),
    color=alt.Color('branche'),
    tooltip=['branche','count', 'rel_count:Q'],
).properties(
    width=150,
    height=150
).transform_calculate(
    rel_count='datum.count / datum.count_all'
).facet(
    facet='fraktion',
    columns=3
).resolve_scale(
    # theta='shared',
    radius='independent',
)
chart.save('output/mehrere-radial-nach-fraktion-und-branche-min20.pdf')
chart

alt.FacetChart(...)

In [302]:
source = duckdb.query('''
    select distinct p.branche, p.parlamentarier, p.partei, p.kanton,
        count(interessenbindung_id) over (partition by p.branche, p.parlamentarier) count,
        count(interessenbindung_id) over (partition by p.branche) count_all
    from (
        select branche, parlamentarier, partei, kanton
        from (select parlamentarier, partei, kanton from df_bezahlt group by 1,2,3 order by count(1) desc limit 12) p
        cross join (select branche from df_bezahlt group by branche order by count(1) desc limit 8) b
    ) p left join df_bezahlt b on (p.branche, p.parlamentarier) = (b.branche, b.parlamentarier)
    where 1=1
    union all
    select distinct 'Übrige', p.parlamentarier, b.partei, b.kanton,
        count(1) over (partition by p.parlamentarier) count,
        count(1) over () count_all
    from (
        select parlamentarier
        from (select parlamentarier as parlamentarier from df_bezahlt group by 1 order by count(1) desc limit 12) p
        cross join (select branche from df_bezahlt group by branche order by count(1) desc offset 8) b
    ) p join df_bezahlt b on (p.parlamentarier) = (b.parlamentarier)
    where branche not in (select branche from df_bezahlt group by branche order by count(1) desc limit 8)
''').to_df()

chart = alt.Chart(source).mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4,
    # tooltip=True
).encode(
    theta=alt.Theta('branche', stack='center', scale=alt.Scale(rangeMin=0)),
    radius=alt.Radius('count', type='quantitative', scale=alt.Scale(type='sqrt', rangeMax=70), stack=False),
    color=alt.Color('branche'),
    tooltip=[alt.Tooltip('branche'), alt.Tooltip('count', title='anzahl'), alt.Tooltip('rel_count', type='quantitative', format='.1%', title='anteil')],
).properties(
    width=150,
    height=150
).transform_calculate(
    rel_count='datum.count / datum.count_all',
    parl_label='datum.parlamentarier + " (" + datum.partei + "/" + datum.kanton + ")"'
).facet(
    facet='parl_label:N',
    columns=3
).resolve_scale(
    # radius='independent',
    radius='independent',
)
chart.save('output/mehrere-radial-nach-parlamentarier-und-branche-top12.pdf')
chart

alt.FacetChart(...)

In [303]:
source = duckdb.query('''
select fraktion, branche, (select count(1) from df_bezahlt where branche = b.branche and fraktion = f.fraktion) count
from (select distinct fraktion from df_bezahlt) as f
cross join (select distinct branche from df_bezahlt) as b
group by 1, 2
''').to_df()

alt.Chart(source).mark_rect().encode(
    x='branche',
    y='fraktion',
    tooltip='count',
    color=alt.Color('count',
        scale=alt.Scale(scheme='yelloworangered', domain=[0,150]),
        legend=alt.Legend(direction='horizontal')
    )
)

alt.Chart(...)

In [304]:
duckdb.query('''
select distinct branche, geschlecht, count(1) over (partition by branche, geschlecht) cnt, 100.0*count(1) over (partition by branche, geschlecht) / count(1) over (partition by branche) perc
from df_bezahlt
order by 4 desc
''').to_df()

,branche,geschlecht,cnt,perc
0,Verkehr,M,71,84.523810
1,Sport,M,11,78.571429
2,Energie,M,39,76.470588
3,Landwirtschaft,M,40,75.471698
4,Wirtschaft,M,247,74.397590
5,Umwelt,M,17,70.833333
6,Bildung,M,24,68.571429
7,Aussenpolitik/Aussenwirtschaft,M,2,66.666667
8,Gesundheit,M,78,66.101695
9,Kommunikation,M,11,64.705882


In [305]:
source = duckdb.query('''
select branche, geschlecht, count(1) anzahl
from df_bezahlt
group by 1, 2
order by 1, 2
''').to_df()

bars = alt.Chart(source).mark_bar().encode(
    x=alt.X('anzahl', stack="normalize", axis=alt.Axis(format='%'), title='anteil'),
    y='branche',
    color='geschlecht:N',
    tooltip='anzahl',
)

text = alt.Chart(source).mark_text(dx=-12, color='white').encode(
    x=alt.X('anzahl', stack="normalize"),
    y='branche',
    detail='geschlecht:N',
    text='anzahl',
)

source2 = duckdb.query('''
select 97.0/246 anteil_frauen
''').to_df()

rule = alt.Chart(source2).mark_rule(color='red').encode(
    x='anteil_frauen'
)

chart = bars + rule
chart.save('output/bars-nach-branche-und-geschlecht-relativ.pdf')
chart

alt.LayerChart(...)

In [306]:
source = duckdb.query('''
    select lobbygruppe lobby, fraktion_lang fraktion, f.sort, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    where 1=1
    and branche = 'Wirtschaft'
    and lobbygruppe in (select lobbygruppe from df_bezahlt group by lobbygruppe having count(1) >= 5)
    group by 1, 2, 3
    union
    select 'Übrige' lobby, fraktion_lang fraktion, sort, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    and branche = 'Wirtschaft'
    where lobbygruppe not in (select lobbygruppe from df_bezahlt group by lobbygruppe having count(1) >= 5)
    group by 1, 2, 3
    order by lobby, f.sort
''').to_df()

chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anzahl', stack='zero'),
    y=alt.Y('lobby', title='Wirtschaft Lobbygruppe'),
    color=alt.Color('fraktion', scale=alt.Scale(domain=fraktion_namen, range=fraktion_farben)),
    order=alt.Order('color_fraktion_sort_index:Q'),
)
chart.save('output/bars-wirtschaft-nach-lobbygruppe-und-fraktion-absolut-min5.pdf')
chart

alt.Chart(...)

In [307]:
source = duckdb.query('''
    select lobbygruppe lobby, fraktion_lang fraktion, f.sort, count(1) count
    from df_bezahlt b
    join df_fraktionen f on f.fraktion = b.fraktion
    where 1=1
    and branche = 'Wirtschaft'
    group by 1, 2, 3
    order by lobby, f.sort
''').to_df()

chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anzahl', stack='zero'),
    y=alt.Y('lobby', title='Wirtschaft Lobbygruppe'),
    color=alt.Color('fraktion', scale=alt.Scale(domain=fraktion_namen, range=fraktion_farben)),
    order=alt.Order('color_fraktion_sort_index:Q'),
)
chart.save('output/bars-wirtschaft-nach-lobbygruppe-und-fraktion-absolut-alle.pdf')
chart

alt.Chart(...)

In [308]:
source = duckdb.query('''
    select distinct p.branche, fraktion_lang fraktion,
        count(1) over (partition by p.branche, p.fraktion) count,
        count(1) over (partition by p.branche) count_all
    from (
        select branche, fraktion
        from (select distinct fraktion from df_bezahlt) p
        cross join (select branche from df_bezahlt group by branche having count(1) >= 20 /*order by count(1) desc limit 9*/) b
    ) p left join df_bezahlt on (p.branche, p.fraktion) = (df_bezahlt.branche, df_bezahlt.fraktion)
    join df_fraktionen f on f.fraktion = p.fraktion
    where 1=1
    union all
    select distinct 'Übrige', fraktion_lang fraktion, count(1) over (partition by p.fraktion), count(1) over ()
    from df_bezahlt p
    join df_fraktionen f on f.fraktion = p.fraktion
    where branche not in (select branche from df_bezahlt group by branche having count(1) >= 20)
''').to_df()

base = alt.Chart().encode(
    theta=alt.Theta('fraktion', stack=True, scale=alt.Scale(domain=fraktion_namen, rangeMin=0)),
    radius=alt.Radius('count:Q', scale=alt.Scale(type='sqrt', rangeMax=75), stack=False),
    color=alt.Color('fraktion', scale=alt.Scale(domain=fraktion_namen, range=fraktion_farben), legend=None),
    tooltip=[alt.Tooltip('fraktion'), alt.Tooltip('count', title='anzahl'), alt.Tooltip('rel_count', type='quantitative', format='.1%', title='anteil')],
).properties(
    width=180,
    height=180
).transform_calculate(
    rel_count='datum.count / datum.count_all',
)

arcs = base.mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4,
    # tooltip=True
)

text = base.mark_text(radiusOffset=10).encode(
    text='label:N'
).transform_calculate(
    # label='datum.rel_count == 0 ? "" : format(datum.rel_count, ".1%")'
    label='datum.count == 0 ? "" : datum.count'
)

for b in pd.unique(source['branche']):
    chart = alt.layer(arcs, text, data=source).transform_filter(
        (datum.branche == b)
    ).properties(
        title=b
    )
    chart.save(f'output/radial-branche-{b.translate(b.maketrans("/", "_"))}-nach-fraktion.pdf')

In [309]:
source = duckdb.query('''
    select distinct p.branche, fraktion_lang fraktion,
        count(b.interessenbindung_id) over (partition by p.branche, p.fraktion) count,
        count(b.interessenbindung_id) over (partition by p.branche) count_all
    from (
        select distinct branche, fraktion
        from (select fraktion from df_bezahlt) p
        cross join (select branche from df_bezahlt group by branche having count(1) >= 20 /*order by count(1) desc limit 9*/) b
    ) p left join df_bezahlt b on (p.branche, p.fraktion) = (b.branche, b.fraktion)
    join df_fraktionen f on f.fraktion = p.fraktion
    where 1=1
    union all
    select distinct 'Übrige', fraktion_lang fraktion, count(1) over (partition by p.fraktion), count(1) over ()
    from df_bezahlt p
    join df_fraktionen f on f.fraktion = p.fraktion
    where branche not in (select branche from df_bezahlt group by branche having count(1) >= 20)
''').to_df()

base = alt.Chart().mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4
).encode(
    # theta=alt.Theta('branche', stack=True),
    # radius=alt.Radius('count:Q', scale=alt.Scale(type='sqrt', rangeMax=75), stack=False),
    theta=alt.Theta('branche', stack='center'),
    radius=alt.Radius('count', type='quantitative', scale=alt.Scale(type='sqrt', rangeMax=70), stack=False),
    color=alt.Color('branche', legend=None),
    tooltip=['branche','count', 'rel_count:Q'],
).properties(
    width=150,
    height=150
).transform_calculate(
    rel_count='datum.count / datum.count_all'
# ).facet(
#     facet='fraktion',
#     columns=3
# ).resolve_scale(
#     # theta='shared',
#     radius='independent',
)

for f in pd.unique(source['fraktion']):
    chart = alt.layer(base, data=source).transform_filter(
        (datum.fraktion == f)
    ).properties(
        title=f
    )
    chart.save(f'output/radial-fraktion-{f.translate(b.maketrans("/", "_"))}-nach-branche.pdf')